# Overall Term Frequencies

With the TEDtalks-all dataset created, we have 1747 talks with which to work. This is a small corpus, and so the usual reasons for shrinking the feature set for the texts do not apply, but as we begin our survey of the contents of the TED talks we wanted to be mindful of standards that had emerged both so that our results would be comparable to the work of others but also so that we could potentially scale up the work here without having to re-think the foundations.

## Summary

In this notebook we load the complete dataset of both TED and TED+ talks. (No TEDx talks.) We use Python's `Sci-Kit Learn` library to create a document - term frequency matrix with a shape of 1747 x 50379. Summing the words to get a total for each word across all talks in the dataset, we then hand-inspect the totals and discover that there is a small list of numbers that recur. We approach mapping those numbers in two trials before returning to the task of getting a clean frequency list with no repetitions or other oddities.

## Imports and Data

In [1]:
# Imports
import pandas as pd, re, csv, nltk
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
# Load the Data
df = pd.read_csv('../output/TEDall_speakers.csv')
df.shape

(1747, 27)

**To Do**: Edit the CSV to remove the vestigial index column at the start of each line. Then use `df.set_index('Talk_ID')`.

## Frequencies

The sole purpose of this notebook is to establish how we are going to elicit our features, our words, from the collection of talks. Thus, the only column we are interested in is the one with the texts of the talks. As we move forward, however, we will want to decide if we are simply going to append ~30,000 columns to a version of the extant CSV or create a separate CSV for each experiment. 

For this first experiment, we will keep it simple, creating two lists, one of the URLs and one of the texts. The URLs are unique, human-friendly identifiers for the talks. (We can, perhaps, make them a bit more friendly by modifying them a bit, subtracting `https://www.ted.com/talks/` from each.)

In [3]:
urls  = df.public_url.tolist()
texts = df.text.tolist()

There are a number of ways to get term frequencies, but **SciKit-Learn**'s `CountVectorizer` is, I think, the way to go, since it will work well with the other vectorizers and models also available in `sklearn`.

In our first experiment, we run `CountVectorizer` unadorned. The default options are: lowercase everything, get rid of all punctuation, make a word out of anything more than two characters long. The only thing that might not be welcome is the splitting of contractions. For now, we will leave things as they are. (Also, please note, no stopwords were used, so we have an unfiltered word list.)

For this current work, we are running `fit()` and `transform()` separately, but since `fit()` just calculates the parameters and saves them as an internal objects state `transform()`  applies the transformation to a particular set of examples (the ones we just fitted), the two operations are usually simply done at the same time as `fit_transform()`.

In [4]:
# If you want to pass options, pass them here:
vec = CountVectorizer( lowercase = True )

# fit the model to the data 
vecs = vec.fit(texts)

# transform the data according to the fitted model
X = vecs.transform(texts)

# see how many features we have
X.shape

(1747, 50379)

This is our baseline.

In [5]:
df_base = pd.DataFrame(X.toarray(), columns = vecs.get_feature_names())

## Frequency Totals per Word

We can total up our columns for each feature (word), which is something we will be doing per year, per gender, per discipline. Here, we take the vector describing a word and sum it. We then pair the sum with the word in a tuple, which we then sort by frequency. 

(I'm doing it this way because it appears to be the way to do it, but it also strikes me that there should be a way to do this within the array itself, or, perhaps, to do it through **pandas**.)

We save the results to a CSV file so that we can hand-check the words: are these the results we expected? (We don't want any weirdness affecting our overall results.) The hand inspection looks good. I didn't see anything in words 4 or above in frequency that looked off. (So, the simplest solution works!) What I did note was the frequency of certain **numbers**: **100**, **12**, etc. This might be worth taking a closer look: are there *power numbers*? (I am thinking here of Alan Dundes' essay on the "power" of three in American culture.)

**To Do**: It would be nice to be able grab all words of a certain frequency, or range of frequencies.

---
**Follow-up**: whenever I attempt some version of
```python
for item in vecs:
    if vecs.vocabulary_.get(item) == 1691:
        print(item)
```
I get **`TypeError: 'CountVectorizer' object is not iterable`**. My best guess, for now, is that we need to use the tuple above to get this information.

---


In [ ]:
# summing up the counts for each word
sum_words = bow.sum(axis=0)

# create a tuple
words_freq = [(word, sum_words[0, idx]) for word, idx in vecs.vocabulary_.items()]

# sort the tuple
words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)

# check the results of our work by printing the top 20 more frequent words
print(words_freq[0:20])

In [ ]:
# with open('../output/word_freq.csv','w') as out:
#     csv_out = csv.writer(out)
#     csv_out.writerow(['word','count'])
#     for row in words_freq:
#         csv_out.writerow(row)

A hand inspection of the output above turned up a couple of interesting issues: that there are parentheticals mixed in with the text of the talks (see 01-Terms-02-Parentheticals) and numbers feature in the talks regularly (see 01-Terms-03-Numbers). 

There are also some interesting tokenization dimensions to a crowd-sourced transcription technology:

| Term    | Freq |
|:---------|---:|
|aa       |12 |
|aaa      |7  |
|aaaa     |2  |
|aaaaa    |1  |
|aaaaaaaah|1  |
|aaaaaaah |1  |
|aaaaaah  |3  |
|aaaah    |2  |
|aaaahhh  |1  |
|aaah     |3  |
|aag      |1  |
|aah      |10 |

There are an equal number of alternate spellings for *shh*. There's also this oddity:

| Term    | Freq |
|:---------|---:|
|FALSE    |115|
|TRUE     |909|

It appears to be simply the occurrences of *true* and *false* but somehow the vectorizer thinks they are Booleans?

In [ ]:
onetext = nltk.Text(re.sub("[^a-zA-Z0-9'()]"," ",'\n'.join(texts)).split())

In [ ]:
onetext.concordance("Marimba")

The rest of the hand-examination turns up no other issues, so the basic vectorization built into `sklearn` appears to be satisfactory, with the only exception being its breaking of contractions at the apostrophe: e.g., *isn't* becomes *isn* and *'t*, with the latter being thrown away as too small -- which means that the indefinite article *a* is also not present in the frequencies. 

## Revised Frequencies without Parentheticals

For more on the work leading up to this regex, see the `01-Terms-02-Parentheticals` notebook: it contains a number of experiments, which were vexed by sklearn's vectorizer only accepting strings as inputs. (While gensim appears to accept texts as lists of words, we are hoping, I think, to keep our code base as simple as possible.) The eventual workaround is to clean the texts of the top 20 parentheticals, joining the list back to a string, and then pushing the result to sklearn. This is less than optimum, but the hack works and appears to be a hack performed elsewhere by others.

The number "2" below in the variables reveals that this was the second experiment (of three). If we could figure out a way to handle removing the items in the list below at the level of the string, we could pass that function to `CounterVectorizer()` as a `preprocessor = that_function`. 

For now, the first thing we have is the list of the top 20 parentheticals, some of which are two words:

In [6]:
parentheticals = [ "\(laughter\)", "\(applause\)", "\(music\)", "\(video\)", 
                  "\(laughs\)", "\(applause ends\)", "\(audio\)", "\(singing\)", 
                  "\(music ends\)", "\(cheers\)", "\(cheering\)", "\(recording\)", 
                  "\(beatboxing\)", "\(audience\)", "\(guitar strum\)", 
                  "\(clicks metronome\)", "\(sighs\)", "\(guitar\)", "\(marimba sounds\)", 
                  "\(drum sounds\)" ]
len(parentheticals)

20

In the cells that follow, we create a test string that has two terms, *laughter* and *applause*, both in the speech of the imaginary speaker as well as in a parenthetical. We want to keep the former and remove the latter.

Then we develop a function, `clean_text`, that goes through the list of terms to be removed as it goes through a text. This is a loop within a loop, and I am not sure if this is the most efficient way to do this, but it works. 

We then test the `clean_text` function on the `test` string and then on one of the talks we know is mostly a performance. The results, at long last, are what we want.

In [7]:
test = """Laughter is the best medicine. (Laughter) 
Hold your applause; I'm not done yet. (Applause ends)"""

In [8]:
def clean_text(rgx_list, text):
    new_text = text
    for rgx_match in rgx_list:
        new_text = re.sub(rgx_match, ' ', new_text, flags=re.IGNORECASE)
    return new_text

In [9]:
print(clean_text(parentheticals, test))

Laughter is the best medicine.   
Hold your applause; I'm not done yet.  


In [10]:
print(clean_text(parentheticals, texts[235]))

  Let's just get started here.    Okay, just a moment.    (Whirring)    All right.   Oh, sorry.           Thank you.       


We now have a working function, but the two arguments of `clean_text` cannot be used in `CountVectorizer`: we are going to use this as a `preprocessor` argument, which takes only one argument itself, the text. For the sake of simplicity, we are simply going to embed the regex list, `parentheticals`, into the function, renaming it `clean_parens`.

With that done, we are going to use our new custom preprocessor in a new instantiation of the `CountVectorizer`, which runs up our term count from 50379 to 58404. 

Locally, the comparisons look good: a term like *laughter* drops from 7374 occurrenaces to 87. Other terms show similar drops in usage. 

In [11]:
def clean_parens(text):
    new_text = text
    for rgx_match in parentheticals:
        new_text = re.sub(rgx_match, ' ', new_text.lower(), flags=re.IGNORECASE)
    return new_text

A survey of the global increase in terms, from 50379 to 58404, turned up some interesting restuls: first, the difference between the two sets of names was 19,891 and it looks like the `CountVectorizer` didn't lowercase a lot of things, even when the `lowercase=True` argument was passed to it. This necessitated a slight workaround, until we understand otherwise, of lowering everything in the regex function, `clean_parens`, as had occurred above.

With that change in place, we vectorize things again, and we get the slightest of drops in features, from 50379 to 50377, which means we are pretty stable. Nice! (A hand inspection reveals that this still includes a lot of named entities, so we are going to raise the threshold for `min_df` from the default 1 to 2 in a moment. First let's explore what our cleaning has wrought:

In [12]:
vec_clean = CountVectorizer(preprocessor = clean_parens)
X_clean = vec_clean.fit_transform(texts)
X_clean.shape

(1747, 50377)

In [13]:
df_clean = pd.DataFrame(X_clean.toarray(), columns = vec_clean.get_feature_names())

In [14]:
term = 'laughter'
f"The baseline count for *{term}* is {df_base[term].sum()}; \
with cleaning it is {df_clean[term].sum()}"

'The baseline count for *laughter* is 7374; with cleaning it is 98'

In [15]:
globalist = set(vec_clean.get_feature_names()) - set(vecs.get_feature_names())

print(len(globalist), globalist)

0 set()


I'm not sure why we aren't seeing 2 things there, but we can explore that another time. Let's adjust our threshold for inclusion in the feature set to 2 documents and see what happens:

In [16]:
vec_new = CountVectorizer(preprocessor = clean_parens, min_df = 2)
X_new = vec_new.fit_transform(texts)
X_new.shape

(1747, 29340)

We will repeat the operation above where we sum our terms and write the results to a CSV file, but this time using `pandas`:

In [17]:
df_new = pd.DataFrame(X_new.toarray(), columns = vec_new.get_feature_names())

In [18]:
sums = df_new.sum()

In [22]:
sums[1000:1005]

affiliations    3
affinity        3
affirm          7
affirmation     4
affirmations    3
dtype: int64

In [24]:
# sums.to_csv('../output/word_freq_clean.csv')

## Creating a Revised Dataset with Term Frequencies



We now have a dataframe whose contents are those of the term frequency matrix for all our texts. First we give it an index using the `urls` list drawn from the base dataframe. Then we save it to a CSV, and finally we merge the base dataframe with the term matrix to produce a dataframe that has everything we need for our exploration of trends. 

As above, where cells write files, or possibly overwrite files, the code is commented out.

In [29]:
df_new_labeled = df_new.set_axis(urls, axis=0)

In [30]:
df_new_labeled.head()

,00,000,0001,000th,001,01,02,05,06,07,...,zooms,zoonotic,zoop,zooplankton,zoos,zuccotti,zuckerberg,zuckerman,zulu,zurich
https://www.ted.com/talks/al_gore_on_averting_climate_crisis,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://www.ted.com/talks/david_pogue_says_simplicity_sells,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://www.ted.com/talks/majora_carter_s_tale_of_urban_renewal,0,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://www.ted.com/talks/ken_robinson_says_schools_kill_creativity,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://www.ted.com/talks/hans_rosling_shows_the_best_stats_you_ve_ever_seen,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
# df_new_labeled.to_csv('../output/talks_all_tf.csv')

In [35]:
df_new_labeled.index.name = 'public_url'

In [36]:
df_new_labeled.index.name

'public_url'

In [37]:
df_merge = pd.merge(df, df_new_labeled, on = 'public_url')

In [38]:
# df_merge.to_csv('../output/talks_all_tf_matrix.csv')